In [4]:
import pandas as pd
import numpy as np

In [5]:
read_df = pd.read_csv('output/embedding_all_sentence.csv')
read_df['sentence_embedding'] = read_df.sentence_embedding.apply(eval).apply(np.array)

In [1]:
import neo4j

host = "bolt://localhost"
username = "neo4j"
password = "1234qwer"

driver = neo4j.GraphDatabase.driver(f'{host}', auth=(username, password))

session = driver.session()

In [6]:
for index, row in read_df.iterrows():
    session.run("MATCH (n:Requirement {id: $id}) SET n.sentence_embedding = $sentence_embedding", id=row['id'], sentence_embedding=row['sentence_embedding'])

In [15]:
records, summary, key = driver.execute_query("""
MATCH (n:Features) WHERE n.app <> $app
WITH n, vector.similarity.cosine(n.sentence_embedding, $embedding) as score
RETURN n.id, n.app, n.sentence, score
ORDER BY score DESC LIMIT 5;
""", embedding=read_df['sentence_embedding'][10], app=read_df['app'][10])

print(read_df['sentence'][10], '\n')

for row in records:
    print(row['n.id'], row['n.app'], row['n.sentence'], row['score'])

Organize your store better by creating menu items. Add pages, links, products, categories, and even collections as menu items. Rearrange the existing menus with ease via drag-and-drop. 

66202056-8904-46c7-a306-09e25d9bec41 Odoo POS Shop Organize your products with hierarchical product categories. Order them by popularity and display different categories at different points of sale. 0.9426832795143127
ee70c104-a47e-4640-8aba-ae1cd95933ae POS Connect Inventory is displayed through the web store according to the theme a client has chosen for their store. Inventory can also be organized and searched for by category and department. 0.9280788898468018
79fcd750-48a2-4194-b5fb-b243aa8639d6 Odoo Purchase Organize the movement of items between two locations you own. 0.9187054634094238
7a5dd008-2209-44d9-a098-96b4f1f81cb6 Odoo POS Shop Find your customers and products with the built-in search features. 0.9182571172714233
c8b7e45f-1c36-490f-85ca-8585ea0adbb5 Odoo Inventory Locate products, their 

In [2]:
import pandas as pd
df_dataset = pd.read_csv('data/datasets_all_sentence.csv')

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

/Users/fahmisyaifudin/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/fahmisyaifudin/miniconda3/envs/llm/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
requirements = df_dataset['sentence'].tolist()
requirement_embedding = model.encode(requirements)

In [8]:
for index, row in df_dataset.iterrows():
    session.run("MATCH (n:Requirement {id: $id}) SET n.sbert_embedding = $embedding", id=row['id'], embedding=requirement_embedding[index])

In [9]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

In [10]:
def bert_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    pooler_output = output['pooler_output'][0]
    return pooler_output.detach().numpy()

In [11]:
for index, row in df_dataset.iterrows():
    embedding = bert_embedding(row['sentence'])
    session.run("MATCH (n:Requirement {id: $id}) SET n.bert_embedding = $embedding", id=row['id'], embedding=embedding)